In [1]:
from __future__ import annotations
from datetime import datetime, timezone
from pathlib import Path

from pymongo.results import InsertOneResult

from twitter_trends import ArgsTwitterTrends, ParamsTwitterTrends, WOEIDCountry
from scraping_kit import load_db_and_bots, BotScraper
from scraping_kit.db.models.raw import RawData
from scraping_kit.db.models.trends import Trends
from scraping_kit.db.db_twitter import DBTwitter
from scraping_kit.utils import date_delta

path_data = Path("data")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

Collection Names: []
Bots: bots=[BotScraper(acc_name='Vera')]


In [2]:
db_tw.db.drop_collection("raw")
db_tw.db.drop_collection("trends")

{'ok': 1.0}

In [3]:
def process_trends_raw_data(db_tw: DBTwitter, insert_one_result_raw: InsertOneResult) -> InsertOneResult:
    raw_data = db_tw.coll.raw.find_one(insert_one_result_raw.inserted_id)
    raw_data = RawData(**raw_data)
    trends = Trends.from_raw(raw_data)
    insert_one_result_trend = db_tw.coll.trends.insert_one(trends.model_dump())     # TODO: Check its ok.
    return insert_one_result_trend

def requests_trends_and_save(db_tw: DBTwitter, req_args: ArgsTwitterTrends, bot: BotScraper) -> InsertOneResult:
    response, creation_date = bot.get_response(req_args)
    insert_one_result_raw = db_tw.add_raw_data(req_args, response, creation_date)
    return insert_one_result_raw

def requests_and_process(db_tw: DBTwitter, req_args: ArgsTwitterTrends, bot: BotScraper) -> InsertOneResult:
    insert_one_result_raw = requests_trends_and_save(db_tw, req_args, bot)
    insert_one_result_trend = process_trends_raw_data(db_tw, insert_one_result_raw)
    db_tw.coll.change_is_processed(insert_one_result_raw)
    return insert_one_result_trend

In [4]:
req_args = ArgsTwitterTrends(data=ParamsTwitterTrends(woeid=WOEIDCountry.united_states))    # You can change woeid if you need.
idx_bot_choiced, bot_choiced = bots.random_bot()

insert_one_result_trend = requests_and_process(db_tw, req_args, bot_choiced)
insert_one_result_trend

InsertOneResult(ObjectId('658c1e608b5058981a2f5119'), acknowledged=True)

In [11]:
date = date_delta(
    date_i = datetime(2023, 12, 27, tzinfo=timezone.utc),
    date_f = datetime(2023, 12, 28, tzinfo=timezone.utc)
)

#trends = db_tw.coll.trends_from_insert_id(insert_one_result_trend)     # --> Search by id in the DB.
trends = db_tw.coll.trends_from_date_range(date)                        # --> Search by date range, returns the first result found in that range.

df_trends = trends.get_df()
df_trends

,name,query,url,volume,domainContext
0,Christmas,Christmas,search?q=Christmas,3480000,
1,Jesus,Jesus,search?q=Jesus,710000,
2,Santa,Santa,search?q=Santa,425000,
3,Boxing Day,Boxing+Day,search?q=%22Boxing+Day%22,253000,
4,Happy Holidays,Happy+Holidays,search?q=%22Happy+Holidays%22,239000,
5,Canserbero,Canserbero,search?q=Canserbero,135000,Rap
6,Happy New Year,Happy+New+Year,search?q=%22Happy+New+Year%22,115000,Holidays
7,Kwanzaa,Kwanzaa,search?q=Kwanzaa,85400,Politics
8,Delta,Delta,search?q=Delta,73600,Travel
9,Merry Xmas,Merry+Xmas,search?q=%22Merry+Xmas%22,70800,
